<a href="https://colab.research.google.com/github/flpsantoro/SPRINT_IV/blob/main/SPRINT_IV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>